In [1]:
# | default_exp routes.user


In [2]:
# | exporti
import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


# SEARCH and GET Routes


## Search Users


In [3]:
# | export
async def get_all_users(
    auth: dmda.DomoAuth, debug_api: bool = False
) -> rgd.ResponseGetData:
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    return await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api
    )

#### sample implementation of get_all_users


In [4]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_all_users(auth=token_auth)
pd.DataFrame(res.response[0:5])

,id,invitorUserId,displayName,userName,emailAddress,accepted,userType,timeZone,modified,created,role,roleId,rights,active,pending,systemUser,anonymous
0,0,2,monitor,monitor,monitor@domo.com,False,DOMO_SUPPORT,Etc/GMT+7,1588877394091,1464820854,Privileged,1,31.0,True,True,True,True
1,1006847540,1893952720,Marc-Anton Clavel,marcanton.clavel@domo.com,marcanton.clavel@domo.com,False,USER,NaN,1665166064000,1618579073,Privileged,2,31.0,True,True,False,True
2,1012895591,1893952720,JeMiller,JeMiller@marketaxess.com,JeMiller@marketaxess.com,True,USER,NaN,1657051684429,1657049419,NaN,2097317660,NaN,True,False,False,False
3,1024352013,1893952720,Michael Shrifteylik,mshrifteylik@sportsresearch.com,mshrifteylik@sportsresearch.com,True,USER,NaN,1669658030028,1667838056,Privileged,2,31.0,True,False,False,False
4,1026684838,1893952720,Ghofrane Yahmadi,ghofrane.yahmadi@audibene.de,ghofrane.yahmadi@audibene.de,False,USER,NaN,1618579110000,1618579109,Privileged,2,31.0,True,True,False,True


In [5]:
# | export
def generate_search_users_body_by_id(user_ids: list[str]) -> dict:
    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "filters": [
            {"field": "id", "filterType": "value",
                "values": user_ids, "operator": "EQ"}
        ],
    }


In [6]:
# | export
def generate_search_users_body_by_email(email_address: str) -> dict:
    """search does not appear to be case sensitive"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "limit": 200,
        "offset": 0,
        "sort": {"field": "displayName", "order": "ASC"},
        "filters": [
            {"filterType": "text", "field": "emailAddress", "text": email_address}
        ],
    }


In [7]:
#| export
def process_v1_search_users_user(obj_user):
    dd_user = dd.DictDot(obj_user)

    return {
        "id": dd_user.id,
        "displayName": dd_user.displayName,
        "roleId": dd_user.roleId,
        "userName": dd_user.userName,
        "emailAddress": dd_user.emailAddress,
    }


def process_v1_search_users(res_user_v1_search: rgd.ResponseGetData):
    obj_users = res_user_v1_search.response.get("users")

    return [process_v1_search_users_user(obj_user) for obj_user in obj_users]

In [8]:
# | export
async def search_users(
    auth: dmda.DomoAuth,
    body: dict,
    debug_api: bool = False,
    process_users: bool = True,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/search"

    res = await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

    if process_users and res.is_success:
        res.response = process_v1_search_users(res)

    return res

#### sample implementation of search_users


In [9]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

search_email_body = generate_search_users_body_by_email(
    email_address="jae@onyxreporting.com"
)

res = await search_users(auth=token_auth, body=search_email_body, process_users=True)
res.response

[{'id': 1893952720,
  'displayName': 'Jae Wilson',
  'roleId': 1,
  'userName': 'jae@onyxreporting.com',
  'emailAddress': 'jae@onyxreporting.com'}]

## Search Publish


In [10]:
#| export
async def search_virtual_user_by_subscriber_instance(
    auth: dmda.DomoAuth,
    subscriber_instance,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/proxy_user/domain/"

    body = {"domains": [f"{subscriber_instance}.domo.com"]}

    return await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

# CRUD Routes and Attributes


In [11]:
# | export
async def create_user(
    auth: dmda.DomoAuth,
    display_name: str,
    email: str,
    role_id: int,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users"

    body = {"displayName": display_name, "detail": {
        "email": email}, "roleId": role_id}

    return await gd.get_data(
        url=url, method="POST", body=body, auth=auth, debug_api=debug_api
    )


In [12]:
# | export
async def reset_password(
    auth: dmda.DomoAuth,
    user_id: str,
    new_password: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/password"

    body = {"domoUserId": user_id, "password": new_password}

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )


In [13]:
# | export
async def request_password_reset(
    domo_instance: str, email: str, 
    locale="en-us", debug_api: bool = False
):
    url = f"https://{domo_instance}.domo.com/api/domoweb/auth/sendReset"

    params = {"email": email, "local": locale}

    return await gd.get_data(
        url=url, method="GET", params=params, auth=None, debug_api=debug_api
    )

In [14]:
# | export
async def set_user_landing_page(
    auth: dmda.DomoAuth, 
    user_id: str, page_id: 
    str, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/landings/target/DESKTOP/entity/PAGE/id/{page_id}/{user_id}"

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        # body = body,
        debug=debug,
    )